whisperはopenaiによって開発された文字起こしAI（speach to text）  
https://huggingface.co/openai/whisper-small  

モデルは学習済のものの内、もっとも容量が軽いものを利用する。

huggingfaceには多くの学習済モデルがアップされており、利用規約の範囲内なら手軽に利用することができる。

In [1]:
# パッケージんストール
!pip install -U openai-whisper setuptools-rust

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for setuptools-rust from https://files.pythonhosted.org/packages/e0/34/d88a7ceb193fbcee6c8992d1b1e33ed20361027e07fea1676efc45ec7a43/setuptools_rust-1.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for triton<3,>=2.0.0 from https://files.pythonhosted.org/packages/4d/22/91a8af421c8a8902dde76e6ef3db01b258af16c53d81e8c0d0dc13900a9e/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/ed/13/b66627125b35f2987bd9872cf028b5e1e1ffcbc8d1e182ac4e84eed3998f/numba-0.58.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for torch from https://files.python

In [4]:
# 音声ファイルをダウンロード

import requests

url='https://pro-video.jp/voice/announce/mp3/001-sibutomo.mp3'
filename='001-sibutomo.mp3'

urlData = requests.get(url).content

with open(filename ,mode='wb') as f: # wb でバイト型を書き込める
  f.write(urlData)

In [14]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("001-sibutomo.mp3")
whisper_result = result["text"]
print(whisper_result)

/usr/local/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


無天下のしゃぼん玉石圏なら もう安心天然の保湿成分が含まれるため 肌にうるおよあたえすこやかにたもちますお肌のことでお悩みの方は ぜひ一度無天下しゃぼん玉石圏をお試しくださいおもとめは01,20,00,5,5,9,5まで


## 結果
あまりうまく聞き取れていない。

原因としては日本の無添加等のドメイン知識を持っていないこと、あるいはモデルが小さいため精度を上げることが難しい、などが挙げられる。

## LLMも試してみよう

日本産LLM: https://github.com/llm-jp/awesome-japanese-llm?tab=readme-ov-file



In [40]:
# 思ってたよりも手軽に動かなかったのでコメントアウト

"""
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("line-corporation/line-distilbert-base-japanese", trust_remote_code=True)
model = AutoModel.from_pretrained("line-corporation/line-distilbert-base-japanese")

sentence = "無添加のしゃぼん玉[MASK]なら安心天然の保湿成分が含まれる"

tokenized_text = tokenizer.tokenize(sentence)
masked_index = tokenized_text.index("[MASK]")
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])

model.eval()

with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)
"""

'\nimport torch\nfrom transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained("line-corporation/line-distilbert-base-japanese", trust_remote_code=True)\nmodel = AutoModel.from_pretrained("line-corporation/line-distilbert-base-japanese")\n\nsentence = "無添加のしゃぼん玉[MASK]なら安心天然の保湿成分が含まれる"\n\ntokenized_text = tokenizer.tokenize(sentence)\nmasked_index = tokenized_text.index("[MASK]")\nindexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)\ntokens_tensor = torch.tensor([indexed_tokens])\n\nmodel.eval()\n\nwith torch.no_grad():\n    outputs = model(tokens_tensor)\n    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出\n\nfor i, index_t in enumerate(predictions.indices):\n    index = index_t.item()\n    token = tokenizer.convert_ids_to_tokens([index])[0]\n    print(i, token)\n'